# Librerias

In [1]:
import pandas as pd
import numpy as np

import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum

import math

#### Conexion Google Drive

In [2]:
#from google.colab import drive
#drive.mount("/content/gdrive")

In [3]:
#from google.colab import auth
#auth.authenticate_user()

#import gspread
#from google.auth import default
#creds, _ = default()

# Clases

## Clase Zona Censal

In [4]:
class Zona_Censal:
    """
    <descripcion>    

    Parameters
    ----------
    x     : float
                coordenada x del centroide de zona censal (ZC)
    y     : float
                coordenada y del centroide de zona censal (ZC)
    n[]** : float
                cantidad de clientes en el segmento en el periodo t
    g[]   : float
                densidad de demanda promedio en el periodo t en [clientes/km^2]
    D[]** : float
                demanda total de la ZC en el periodo t en [items]
    k     : float
                factor de proporcionalidad métrica de distancia
    c     : float
                velocidad local promedio (average local circuity) en [km/hr]
    A     : float
                área de la zona censal en [km^2]
    ID    : int
                identificador de la zona censal (único)
    ds[]  : float
                average drop size in period t [items/cliente]
    T     : int
                periodos

    Other Attributes
    ----------
    loc : tuple
                ubicación combinada (x,y) del centroide de ZC   
    """
        
    def __init__(self,
                iD,
                x,
                y,
                n,
                demanda,
                c,
                area,
                T
                ):
        
        self.ID = int(iD)
        self.x = float(x)
        self.y = float(y)
        self.loc = (self.x,self.y)
        self.activo = [ False if demanda[t] == 0 else True for t in range(T)]
        self.n = np.array(n)
        self.D = np.array(demanda)
        self.g = np.array(n) / area 
        self.c = c
        self.k = 0.57 # Daganzo (1994) L2 norm VRP
        self.A = area
        self.ds = [2]*T
        self.T = T


## Clase Micro Hub


In [5]:
class Micro_Hub:
    """
    <descripción>

    Parameters
    ----------
    x         : float
                coordenada X de la ubicación del MH
    y         : float
                coordenada Y de la ubicación del MH
    C_F[]     : float
                costo fijo de instalar el MH en el periodo t, en [$]
    C_O[t][a] : float
                costo de operar con capacidad "a" en el periodo t, en [$]
    Q{}       : float
                capacidad "a" disponible para MH en [items]
    Q_ID[a]   :str
                lista de capacidades disponibles para MH
    ID        : int
                a (preferrably unique) identifier of the micro-hub
                
    Other Attributes
    ----------------
    loc : tuple
                ubicación combinada (x,y) del centroide del MH

    """
    def __init__(self,
                iD,
                x,
                y,
                cf,
                co,
                q,
                q_id,
                T):
        self.ID = int(iD)
        self.x = float(x)
        self.y = float(y)
        self.loc = (self.x,self.y)
        self.ID = int(iD)
        self.C_F = np.array(cf)
        self.C_O = np.array(co)
        self.Q = q
        self.Q_ID = np.array(q_id)
        self.T = T

## Clase Vehiculo

In [6]:
class Vehiculo: 
    """
    <descripción>

    Parameters
    ----------
    Q_V : float
              load capacity of vehicle in [items]
    cd  : float ***
              costo operacional por distancia del vehiculo en [$/km]
    ch  : float ***
              costo operacional por tiempo del vehiculo en [$/hr]
    tp  : float
              preparation time required to dispatch vehicle in [hr]
    cs  : float ***
              wage cost per hour for using vehicle in [$/hr]
    tl  : float
              fixed loading time required in [hr]
    k   : float
              circuit factor for vehicle
    sl  : float
              linehaul speed en [km/hr]
    ts  : float
              route setup time (Fixed set-up time) en [hr]
    s   : float
              intra-stop speed of vehicles of type en [km/hr]


    Tm  : float
              maximum allowed service time (per vehicle) in [hr]
    td  : float
              service time per stop in [hr]
    F   : float
              costo fijo de usar el vehiculo
    ID  : int
              a (preferrably unique) identifier of the vehicle type
    """
    
    def __init__(self,
                 Q_V,
                 cd,
                 ch,
                 tp,
                 cs,
                 tl,
                 sl,
                 ts,
                 s,
                 Tm,
                 td,
                 F,
                 ID):
        self.Q_V = float(Q_V) 
        self.cd = float(cd) 
        self.ch = float(ch)
        self.tp = float(tp)
        self.cs = float(cs) 
        self.tl = float(tl) 
        self.k = float(1) 
        self.sl = float(sl) 
        self.ts = float(ts)
        self.s = float(s)
        self.Tm = float(Tm)
        self.td = float(td)
        self.F = float(F)
        self.ID = ID

# Cargar Data

Previamente GENERADA: ZC, MH, distancias

## Segmentos

In [7]:
# cargar ZC
#path_zc = '/content/gdrive/MyDrive/Multi-Period 2E-LRP/Instancias_Parametros/Base ZC.csv'
path_zc = r"C:\\Users\user\Desktop\Investigacion JC - PH\Data In\Base ZC.csv"
df_zc = pd.read_csv(path_zc)
display(df_zc)

,Unnamed: 0,GEOCODIGO,REGION,NOM_REGION,PROVINCIA,NOM_PROVIN,COMUNA,NOM_COMUNA,URBANO,DISTRITO,...,demanda1,customer1,demanda2,customer2,demanda3,customer3,demanda4,customer4,demanda5,customer5
0,0,13101011001,13,REGIÓN METROPOLITANA DE SANTIAGO,131,SANTIAGO,13101,SANTIAGO,SANTIAGO,1,...,165,54,229,63,237,70,118,74,160,54
1,1,13101011002,13,REGIÓN METROPOLITANA DE SANTIAGO,131,SANTIAGO,13101,SANTIAGO,SANTIAGO,1,...,106,51,243,57,173,95,291,75,266,77
2,2,13101011003,13,REGIÓN METROPOLITANA DE SANTIAGO,131,SANTIAGO,13101,SANTIAGO,SANTIAGO,1,...,184,72,113,82,119,131,139,65,129,51
3,3,13101011004,13,REGIÓN METROPOLITANA DE SANTIAGO,131,SANTIAGO,13101,SANTIAGO,SANTIAGO,1,...,215,62,110,50,144,68,111,66,164,66
4,4,13101011005,13,REGIÓN METROPOLITANA DE SANTIAGO,131,SANTIAGO,13101,SANTIAGO,SANTIAGO,1,...,166,65,139,89,172,56,116,73,143,85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1642,1642,13604041002,13,REGIÓN METROPOLITANA DE SANTIAGO,136,TALAGANTE,13604,PADRE HURTADO,PADRE HURTADO,4,...,156,51,145,54,124,56,170,65,102,76
1643,1643,13604041003,13,REGIÓN METROPOLITANA DE SANTIAGO,136,TALAGANTE,13604,PADRE HURTADO,PADRE HURTADO,4,...,112,100,126,56,184,81,116,105,113,60
1644,1644,13604041004,13,REGIÓN METROPOLITANA DE SANTIAGO,136,TALAGANTE,13604,PADRE HURTADO,PADRE HURTADO,4,...,126,52,130,64,224,68,142,60,105,91
1645,1645,13604041005,13,REGIÓN METROPOLITANA DE SANTIAGO,136,TALAGANTE,13604,PADRE HURTADO,PADRE HURTADO,4,...,114,85,157,74,111,57,116,54,167,70


### Generar Lista ZC

In [8]:
List_ZC = []

for lab, row in df_zc.iterrows():
    n = [row['customer1'],row['customer2'],row['customer3'],row['customer4'],row['customer5']]
    demanda = [row['demanda1'],row['demanda2'],row['demanda3'],row['demanda4'],row['demanda5']]

    new_segment = Zona_Censal(row['GEOCODIGO'],
                              row['lon'],
                              row['lat'],
                              n,
                              demanda,
                              row['Vel_promedio'],
                              row['area_km2'],
                              2)
    List_ZC.append(new_segment)
print("Zonas Censales: ",len(List_ZC))

Zonas Censales:  1647


## MicroHub

In [9]:
# cargar MH
#path_mh = '/content/gdrive/MyDrive/Multi-Period 2E-LRP/Instancias_Parametros/Base MH.csv'
path_mh = r"C:\Users\user\Desktop\Investigacion JC - PH\Data In\Base MH.csv"
df_mh = pd.read_csv(path_mh)
display(df_mh)

,Unnamed: 0,osm_id,name,access,addr.city,addr.housenumber,addr.street,amenity,barrier,building,...,co1_large,co2_large,co3_large,co4_large,co5_large,cf1,cf2,cf3,cf4,cf5
0,0,144015370,Centro Comercial La Reina,yes,NaN,NaN,NaN,parking,NaN,NaN,...,1784,1686,2445,1776,2740,10400,13637,46986,10067,12100
1,1,183347000,Municipalidad de Independencia,yes,NaN,NaN,NaN,parking,NaN,NaN,...,2283,3915,1622,1817,2146,28700,10848,12948,16176,10016
2,2,255805128,NaN,yes,Pudahuel,NaN,Avenida Teniente Cruz,parking,NaN,NaN,...,1690,1707,2316,2135,2687,19469,16984,23535,13312,11450
3,3,255805130,NaN,yes,NaN,NaN,NaN,parking,NaN,NaN,...,1832,2220,2247,2426,1745,15424,16764,14994,11574,10530
4,4,255805131,NaN,yes,Pudahuel,NaN,Avenida Teniente Cruz,parking,NaN,NaN,...,2562,1752,1629,1794,1633,13344,11400,14091,11533,13225
5,5,255805132,NaN,yes,Pudahuel,NaN,Avenida Teniente Cruz,parking,NaN,NaN,...,3150,3374,1649,2166,1949,15696,11621,11579,20185,23766
6,6,272112665,NaN,yes,NaN,3233.0,Dublé Almeyda,parking,wall,NaN,...,1614,2378,2359,4221,1687,14730,12039,12874,13048,10519
7,7,273273842,NaN,yes,NaN,NaN,NaN,parking,NaN,NaN,...,1611,1604,3287,1873,2060,10238,16413,10161,30308,21823
8,8,278048435,NaN,yes,NaN,NaN,NaN,parking,NaN,NaN,...,2167,1757,2295,1847,1690,10317,11274,14111,20379,16398
9,9,292333297,NaN,yes,NaN,NaN,NaN,parking,NaN,NaN,...,2192,1601,1923,1673,1740,11864,17598,12006,10615,14285


### Capacidad

In [15]:
# ********* google colab *************
#gc = gspread.authorize(creds)
#worksheet = gc.open('Base MicroHub').sheet1
# get_all_values gives a list of rows.
#rows = worksheet.get_all_values()

# Convert to a DataFrame and render.
#df_mh_q = pd.DataFrame.from_records(rows)
#df_mh_q.columns = df_mh_q.iloc[0,:]
#df_mh_q.drop(0,inplace=True)


# ********* LOCAL **************
path_mh_q = r"C:\Users\user\Desktop\Investigacion JC - PH\Data In\Base MicroHub.xlsx"
df_mh_q = pd.read_excel(path_mh_q)
display(df_mh_q)

,osm_id,144015370,183347000,255805128,255805130,255805131,255805132,272112665,273273842,278048435,...,891116578,907134697,909674530,922756663,941519793,953603864,964467872,972434583,983415168,1033557392
0,small,10326,3754,3608,10601,11796,1885,8152,777,4745,...,13371,9314,13883,4723,4023,16036,9663,8817,6501,8571
1,medium,13969,6285,6695,17959,12075,3709,17049,2813,6695,...,2211,14333,11480,16437,13014,8468,19089,15471,8920,11307
2,large,14932000,17094000,17163000,19155000,12607000,14503000,17701000,16796000,12961000,...,11249000,9743000,824000,2633000,12680000,19304000,1605000,19516000,12795000,16908000


In [24]:
df_mh_q[144015370]

0       10326
1       13969
2    14932000
Name: 144015370, dtype: int64

### Generar Lista de *MH*

In [28]:
List_MH = []

for lab, row in df_mh.iterrows():
    # costo fijo
    cf = [row['cf1'],row['cf2'],row['cf3'],row['cf4'],row['cf5']]
    # costo operar - fila periodo y columna capacidad
    co = [[row['co1_small'],row['co1_medium'],row['co1_large']],
          [row['co2_small'],row['co2_medium'],row['co2_large']],
          [row['co3_small'],row['co3_medium'],row['co3_large']],
          [row['co4_small'],row['co4_medium'],row['co4_large']],
          [row['co5_small'],row['co5_medium'],row['co5_large']]]
    
    # capacidad
    q_id = [0,1,2]
    
    # ********** google colab ************
    #q = {0: df_mh_q[str(row['osm_id'])][0],
    #     1: df_mh_q[str(row['osm_id'])][1],
    #     2: df_mh_q[str(row['osm_id'])][2]}
    # ********** LOCAL ***************
    q = {0: df_mh_q[row['osm_id']][0],
         1: df_mh_q[row['osm_id']][1],
         2: df_mh_q[row['osm_id']][2]}
    
    # creacion de microhub
    new_micro_hub = Micro_Hub(row['osm_id'],
                              row['longitude'],
                              row['latitude'],
                              cf,
                              co,
                              q,
                              q_id,
                              5)
    List_MH.append(new_micro_hub)
    #print(lab,q)
print("Micro Hub: ",len(List_MH))

Micro Hub:  46


## Vehiculo 2E

In [29]:
# ******* google colab ************
#gc = gspread.authorize(creds)
#worksheet = gc.open('Base Vehiculos').sheet1
# get_all_values gives a list of rows.
#rows = worksheet.get_all_values()
# Convert to a DataFrame and render.
#df_v = pd.DataFrame.from_records(rows)
#df_v.columns = df_v.iloc[0,:]
#df_v.drop(0,inplace=True)

# ********** LOCAL ***************
path_v = r"C:\Users\user\Desktop\Investigacion JC - PH\Data in\Base Vehiculos.xlsx"
df_v = pd.read_excel(path_v)
display(df_v)

,id,capacidad,cd,ch,tp,cs,tl,k,sl,ts,s,TM,td,F
0,bicicleta,15000,10,0.1,0.25,2,0.1,1,15,0.1,0.1,8,0.05,5000
1,van_electrica,50000,100,0.1,0.50,5,0.5,1,30,0.5,30.0,8,0.05,50000


### Generar Lista Vehiculos 2E


In [30]:
List_VH = []

for lab,row in df_v.iterrows():
  # creacion de vehiculo
  new_vehicle = Vehiculo(row['capacidad'],
                          row['cd'],
                          row['ch'],
                          row['tp'],
                          row['cs'],
                          row['tl'],
                          row['sl'],
                          row['ts'],
                          row['s'],
                          row['TM'],
                          row['td'],
                          row['F'],
                          row['id'])

  List_VH.append(new_vehicle)
print("Tipos de Vehiculos: ",len(List_VH))

Tipos de Vehiculos:  2


## Matriz Distancia

In [207]:
# cargar matriz distancia
#path_matriz_distancia = '/content/gdrive/MyDrive/Multi-Period 2E-LRP/Instancias_Parametros/Datos hacia ZC sin autopista(pesimista).csv'

path_matriz_distancia = r""
df_distancia = pd.read_csv(path_matriz_distancia)
#display(df_distancia.head())

In [208]:
## diccionario de distancia entre MH-ZC - LINEHAUL DISTANCE FROM MH TO CENTROID OF ZC

distance_mh_zc = dict([((df_distancia.OSM_ID[i],df_distancia.GEOCODIGO[i]),df_distancia.loc[i,"distance.value"]/1000 ) for i in range(len(df_distancia))])

## diccionario de tiempo entre MH-ZC - 

time_mh_zc = dict([((df_distancia.OSM_ID[i],df_distancia.GEOCODIGO[i]),df_distancia.loc[i,"duration.value"]/3600 ) for i in range(len(df_distancia))])

## diccionario de tiempo con TRAFICO entre MH-ZC

timeTraffic_mh_zc = dict([((df_distancia.OSM_ID[i],df_distancia.GEOCODIGO[i]),df_distancia.loc[i,"duration_in_traffic.value"]/3600 ) for i in range(len(df_distancia))])


# Funciones

## Función ARCE-Multiperiodo:

In [209]:
def arce(zona_censal, micro_hub, vehiculo, distancia, tiempo, t):
    """
    Calcula la estimación del costo de la ruta aumentada para un segmento dado, considerando un vehiculo y un MH para un periodo t especifico
            
    ---------
    result     : dict
        dictionnary conttaining total cost and all intermediate results like: 
    """  
    h       = 0   # paradas por ruta considerando solo CAPACIDAD-vehicular
    T_F     = 0   # duracion fija de cada ruta
    T_c     = 0   # duracion variable por cliente de cada ruta
    n       = 0   # numero real de paradas
    m       = 0   # numeor de rutas por vehiculo por dia
    w       = 0   # tamaño de flota
    f_F     = 0   # costo fijo de vehiculo
    f_D     = 0   # costo de la ruta considerando la distancia
    f_T     = 0   # costo de la ruta considerando el tiempo
    f       = 0   # costo total de la ruta
    
    result  = []   # TOTAL COSTO


    if (zona_censal.n[t] != 0 or zona_censal.D[t]!=0):
      if zona_censal.D[t] < zona_censal.n[t]:
        new_num_stop = zona_censal.D[t]
      else:
        new_num_stop = zona_censal.n[t]
                    
      # Stops per route accounting for capacity only 
      h = (vehiculo.Q_V /(zona_censal.D[t]/new_num_stop))
              
      # Fixed duration of each route
      T_F = (vehiculo.ts + 2*(distancia[(micro_hub.ID, zona_censal.ID)])/vehiculo.sl)

      # Variable duration per customer of each route
      T_c = (zona_censal.c * zona_censal.k / (math.sqrt(zona_censal.g[t]) * vehiculo.s) + vehiculo.td)

      # Effective number of stops per route
      if h * T_c  + T_F  <= vehiculo.Tm:
        n = (h)
      else:
        n = ((vehiculo.Tm - T_F)/T_c)

      # Number of routes per vehicle per day
      if h * T_c + T_F <= vehiculo.Tm:
        m = (vehiculo.Tm / (h*T_c + T_F))
      else:
        m = (1)

      # Fleet size
      w = ((zona_censal.g[t] * zona_censal.A)/ (n * m))

      # Fixed cost of vehicles
      f_F = (vehiculo.F * w)

      # Distance-based cost of routes
      f_D = (vehiculo.cd * w * m * (2* (distancia[(micro_hub.ID, zona_censal.ID)]) + n * zona_censal.k / math.sqrt(zona_censal.g[t])))

      # Time-based cost of routes
      f_T = (vehiculo.ch * w * m * (vehiculo.ts + 2*(distancia[(micro_hub.ID, zona_censal.ID)]) / vehiculo.sl + n * (vehiculo.td + zona_censal.c * zona_censal.k / (math.sqrt(zona_censal.g[t]) * vehiculo.s))))

      # Total cost of routes
      f = (f_F + f_D + f_T)

      result = {}
      result["h"]   = h
      result["T_F"] = T_F
      result["T_c"] = T_c
      result["n"]   = n
      result["m"]   = m
      result["w"]   = w
      result["f_F"] = f_F
      result["f_D"] = f_D
      result["f_T"] = f_T
      result["f"]   = f

    else:
      result["h"]   = 0
      result["T_F"] = 0
      result["T_c"] = 0
      result["n"]   = 0
      result["m"]   = 0
      result["w"]   = 0
      result["f_F"] = 0
      result["f_D"] = 0
      result["f_T"] = 0
      result["f"]   = 0
    #print(zona_censal.ID, micro_hub.ID, vehiculo.ID,t)
    return result        

# Parámetros

In [210]:
# cantidad de periodos
T = 2

In [211]:
ARCE = dict([ ( (s.ID,m.ID,v.ID,t), arce(s,m,v,distance_mh_zc,time_mh_zc,t) ) for t in range(T) for v in List_VH for m in List_MH for s in List_ZC ])

# Modelo Determinístico

## Full Model

In [216]:
def generarFullModelo():
    modelo = gp.Model("Full Modelo")
    
    # VARIABLES
    
    # Y[m,t] == 1 si se abre el micro-hub "m" en el periodo "t"
    Y = dict([((m.ID,t),modelo.addVar(vtype=GRB.BINARY, name="Y_%s_%s" %(m.ID,t))) for t in range(T) for m in List_MH ])
    
    # W[m,a,t] == 1 si el micro-hub abierto en "m" opera con capacidad "a" en el periodo "t"
    W = dict([ ((m.ID,a,t),modelo.addVar(vtype=GRB.BINARY, name="W_%s_%s_%s" %(m.ID,a,t))) for t in range(T) for m in List_MH for a in m.Q_ID ])
 
    # Z[m,s,v,t] == 1 si el micro-hub "m" sirve a zona censal "s" con vehiculo "v" en el periodo "t"
    Z = dict([((m.ID,s.ID,v.ID,t),modelo.addVar(vtype=GRB.CONTINUOUS, lb=0.0, name="Z_%s_%s_%s_%s" %(m.ID,s.ID,v.ID,t))) for t in range(T) for m in List_MH for s in List_ZC for v in List_VH])

    
    # FUNCION OBJETIVO
    COSTO_FIJO = quicksum( [m.C_F[t]*Y[(m.ID,t)] for t in range(T) for m in List_MH ])
    COSTO_OPERACION = quicksum( [m.C_O[t][a]*W[(m.ID,a,t)] for t in range(T) for m in List_MH for a in m.Q_ID])
    COSTO_TRANSPORTE = quicksum([ARCE[(s.ID,m.ID,v.ID,t)]["f"]*Z[(m.ID,s.ID,v.ID,t)] for t in range(T) for v in List_VH for m in List_MH for s in List_ZC])
    
    Costo_Total = COSTO_FIJO + COSTO_OPERACION + COSTO_TRANSPORTE
        
    modelo.setObjective(Costo_Total,GRB.MINIMIZE)
    
    # RESTRICCIONES
    # (2) capacidad de MH
    for t in range(T):
      for m in List_MH:
        #for a in m.Q_ID:
        r = "R2_t"+str(t)+"_m"+str(m.ID)+"_a"+str("a")
        #display(m.Q[a])
        modelo.addConstr(quicksum([Z[(m.ID,s.ID,v.ID,t)] for s in List_ZC for v in List_VH]) <= quicksum([m.Q[a]*W[(m.ID,a,t)] for a in m.Q_ID]) , name=r)
     # (3) satisfacer demanda
    for t in range(T):
      for s in List_ZC:
        r = "R3_t"+str(t)+"_s"+str(s.ID)
        modelo.addConstr(quicksum([Z[(m.ID,s.ID,v.ID,t)] for m in List_MH for v in List_VH]) >= s.D[t] , name=r)
    
    # (4) apertura de micro-hub
    for m in List_MH:
      r = "R4_m"+str(m.ID)
      modelo.addConstr( quicksum( [Y[(m.ID,t)] for t in range(T) ]) <= 1, name=r )                       
    
    # (5) opera si se ha aperturado antes          
    for t in range(T):
      for m in List_MH:
        r = "R5_t"+str(t)+"_m"+str(m.ID)
        modelo.addConstr( quicksum( [W[(m.ID,a,t)] for a in m.Q_ID] ) <= quicksum( [Y[(m.ID,k)] for k in range(t+1)] ) , name=r)
                       
    
    # EJECUCION MODELO
    
    modelo.optimize()
    
       # mostrar resultados
    print("FO: " + str(Costo_Total.getValue()))
    print("Costo MH: " + str(COSTO_OPERACION.getValue()))
    print("Costo Vehiculo: " + str(COSTO_TRANSPORTE.getValue()))

    for t in range(T): 
      for m in List_MH:
        for a in m.Q_ID:
          if W[m.ID,a,t].x !=0:
            print("periodo: " +str(t) + " MH:" +str(m.ID) + " q:" + str(a))

In [217]:
generarFullModelo()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 240 rows, 1288 columns and 2622 nonzeros
Model fingerprint: 0xe6b2358c
Variable types: 920 continuous, 368 integer (368 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [1e+03, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 3.452564e+08
Presolve removed 45 rows and 664 columns
Presolve time: 0.01s
Presolved: 195 rows, 624 columns, 1230 nonzeros
Variable types: 460 continuous, 164 integer (164 binary)

Root relaxation: objective 6.891283e+07, 27 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 6.8913e+07    0   17 3.4526e+08 6.8913e+07  80.0%     -    0s
H    0     0              

# Descomposición Benders

## Problema Maestro

## Sub Problema

## Algoritmo Resolución